In [1]:
import pandas as pd

In [50]:
df = pd.read_csv('../stats/release2.0_cleaned_stats_mr/stats-1.tsv', sep='\t', 
                 header=None, names=['lang','segments','wcwords','chars','docs'])
print(len(df))

191


In [51]:
linf = pd.read_csv('../stats/_langs/langs_HPLTv2.tsv', sep='\t')
linf

,v2 Language Code (ISO 693-3+script),Language Name,ISO693-3 code,ISO693-3 code macro,ISO693-1 direct code,ISO693-1 through macro
0,ace_Arab,Achinese,ace,NaN,NaN,NaN
1,ace_Latn,Achinese,ace,NaN,NaN,NaN
2,afr_Latn,Afrikaans,afr,NaN,af,af
3,als_Latn,Tosk Albanian,als,sqi,NaN,sq
4,amh_Ethi,Amharic,amh,NaN,am,am
...,...,...,...,...,...,...
188,yue_Hant,Yue Chinese,yue,zho,NaN,zh
189,zho_Hans,Chinese,zho,NaN,zh,zh
190,zho_Hant,Chinese,zho,NaN,zh,zh
191,zsm_Latn,Standard Malay,zsm,msa,NaN,ms


In [53]:
mdf = df.merge(linf, left_on='lang', right_on='v2 Language Code (ISO 693-3+script)', how='left')

In [124]:
tdf = mdf.sum().to_frame().transpose()
tdf.iloc[:,0] = 'TOTAL'
tdf.iloc[:,5:] = ''
for c in tdf.columns[1:5]:
    tdf[c] = tdf[c].astype(float)
tdf.dtypes


/tmp/ipykernel_14142/2693812801.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tdf = mdf.sum().to_frame().transpose()


lang                                    object
segments                               float64
wcwords                                float64
chars                                  float64
docs                                   float64
v2 Language Code (ISO 693-3+script)     object
Language Name                           object
ISO693-3 code                           object
dtype: object

In [125]:
# using https://www.tablesgenerator.com/markdown_tables, load hf_lang_table.tsv and convert to markdown,
# then copy-paste into README.md
pd.concat([tdf,mdf], ignore_index=True).to_csv('hf_lang_table.tsv', float_format='%.2e',sep='\t')
!cat hf_lang_table.tsv

	lang	segments	wcwords	chars	docs	v2 Language Code (ISO 693-3+script)	Language Name	ISO693-3 code	ISO693-3 code macro	ISO693-1 direct code	ISO693-1 through macro
0	TOTAL	3.00e+11	5.56e+12	3.74e+13	1.06e+10						
1	ace_Arab	1.17e+02	8.36e+03	4.97e+04	1.60e+01	ace_Arab	Achinese	ace			
2	ace_Latn	2.06e+05	8.20e+06	5.08e+07	1.29e+04	ace_Latn	Achinese	ace			
3	afr_Latn	3.77e+07	1.00e+09	5.95e+09	1.46e+06	afr_Latn	Afrikaans	afr		af	af
4	als_Latn	9.51e+07	2.71e+09	1.61e+10	5.38e+06	als_Latn	Tosk Albanian	als	sqi		sq
5	amh_Ethi	7.01e+06	1.96e+08	1.03e+09	2.96e+05	amh_Ethi	Amharic	amh		am	am
6	ara_Arab	2.20e+09	4.81e+10	2.80e+11	8.27e+07	ara_Arab	Arabic	ara		ar	ar
7	asm_Beng	2.68e+06	7.34e+07	4.76e+08	1.76e+05	asm_Beng	Assamese	asm		as	as
8	ast_Latn	7.43e+06	1.95e+08	1.24e+09	2.73e+05	ast_Latn	Asturian	ast			
9	awa_Deva	1.32e+05	6.05e+06	2.88e+07	7.28e+03	awa_Deva	Awadhi	awa			
10	ayr_Latn	1.88e+05	3.07e+06	2.51e+07	9.22e+03	ayr_Latn	Central Aymara	ayr	aym		ay
11	azb_Arab	2.39e+06	3.96e+07	2.60

In [43]:
# Based on https://huggingface.co/languages it seems that HF uses only 2-letter codes for more frequent langs, 
# and 3-letter codes for infrequent ones; for some langs both versions exist but 2-letter codes seems to be
# the major ones. 
# We will use 2-letter codes when they exist, otherwise 3-letter codes.
hf_codes = mdf['ISO693-3 code'].where(mdf['ISO693-1 direct code'].isnull(), mdf['ISO693-1 direct code'])
hf_codes[hf_codes.isnull()]

Series([], Name: ISO693-3 code, dtype: object)

In [42]:
# different original codes may be mapped to the same code
print(len(hf_codes), hf_codes.nunique())
hf_codes.value_counts().head(10)

191 186


ace    2
zh     2
ks     2
bjn    2
knc    2
pl     1
oc     1
ory    1
pag    1
pa     1
Name: ISO693-3 code, dtype: int64

In [40]:
# Copy and paste this to README.me, section "language:"
print('\n'.join(f'- {c}' for c in hf_codes.unique()))

- ace
- af
- als
- am
- ar
- as
- ast
- awa
- ayr
- azb
- azj
- ba
- bm
- ban
- be
- bem
- bn
- bho
- bjn
- bo
- bs
- bug
- bg
- ca
- ceb
- cs
- cjk
- ckb
- crh
- cy
- da
- de
- dik
- dyu
- dz
- el
- en
- eo
- et
- eu
- ee
- fo
- fj
- fi
- fon
- fr
- fur
- fuv
- gaz
- gd
- ga
- gl
- gn
- gu
- ht
- ha
- he
- hi
- hne
- hr
- hu
- hy
- ig
- ilo
- id
- is
- it
- jv
- ja
- kab
- kac
- kam
- kn
- ks
- ka
- kk
- kbp
- kea
- khk
- km
- ki
- rw
- ky
- kmb
- kmr
- knc
- kg
- ko
- lo
- lij
- li
- ln
- lt
- lmo
- ltg
- lb
- lua
- lg
- luo
- lus
- lvs
- mag
- mai
- ml
- mr
- min
- mk
- mt
- mni
- mos
- mi
- my
- nl
- nn
- nb
- npi
- nso
- nus
- ny
- oc
- ory
- pag
- pa
- pap
- pbt
- pes
- plt
- pl
- pt
- prs
- quy
- ro
- rn
- ru
- sg
- sa
- sat
- scn
- shn
- si
- sk
- sl
- sm
- sn
- sd
- so
- st
- es
- sc
- sr
- ss
- su
- sv
- swh
- szl
- ta
- taq
- tt
- te
- tg
- tl
- th
- ti
- tpi
- tn
- ts
- tk
- tum
- tr
- tw
- ug
- uk
- umb
- ur
- uzn
- vec
- vi
- war
- wo
- xh
- ydd
- yo
- yue
- zh
- zsm
- zu